https://docs.opencv.org/2.4/doc/tutorials/imgproc/imgtrans/warp_affine/warp_affine.html#warp-affine

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
from copy import deepcopy
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter
from torchvision.transforms import ToPILImage
from PIL import Image
from torchvision.utils import make_grid
from torchvision.transforms import ToTensor
from torch import Tensor

In [ ]:
from biomedical_image_segmentation.plot_utils import insert_grid
from biomedical_image_segmentation.elastic_deform import getAffineTransform

In [ ]:
h, w = 200, 250

## Pick 3 points in image space

In [ ]:
from typing import List
def scatter_plot(
    list_pts: List[np.ndarray], 
    h: int, 
    w: int, 
    extend_xylim_by: int=50):
    
    markers = [
        ".",
        ",",
        "o", 
        "v",
        "^",
        "<",
        ">",
        "1",
        "2",
        "3",
        "4",
        "8",
        "s",
        "p",
        "P",
        "*",
        "h",
        "H",
        "+",
        "x",
        "X",
        "D",
        "d",
        "|"]
    n = len(list_pts)
    fig, axes = plt.subplots(figsize=(4*n, 6), ncols=n)
    
    if not hasattr(axes, "flatten"):
        axes = [axes]
        
    for i, pts in enumerate(list_pts):
        axes[i].scatter(pts[:,0], pts[:,1], marker=markers[i%n], color="green")
        axes[i].set_ylim(-extend_xylim_by, h+extend_xylim_by)
        axes[i].set_xlim(-extend_xylim_by, w+extend_xylim_by)

        axes[i].grid()
#     for (x1, y1), (x2, y2) in zip(pts_src, pts_dst):
#         label = f"{x1:.1f},{y1:.1f}"

#         ax.annotate(
#             label, # this is the text
#             (x1, y1), # this is the point to label
#             textcoords="offset points", # how to position the text
#             xytext=(0, 10), # distance from text to points (x,y)
#             ha='center') # horizontal alignment can be left, right or center
        
#         label = f"{x2:.1f},{y2:.1f}"
#         ax.annotate(
#             label, # this is the text
#             (x2, y2), # this is the point to label
#             textcoords="offset points", # how to position the text
#             xytext=(0, 10), # distance from text to points (x,y)
#             ha='center') # horizontal alignment can be left, right or center

        
    plt.show()

In [ ]:
# find center of image
ref_ratio = 4.
center_coord = np.float32([w, h]) // 2
print("Center coordinates in image space : ", center_coord)

ref_point = min((h, w)) // ref_ratio # reference point
print("Reference point in image space : ", ref_point)

# Affine tranformation will generate transformation matrix when provided with 3 points in original image and 3 points in transformed image
pts_src = np.float32([
    center_coord - ref_point,
    center_coord + np.array([1., -1.]) * ref_point,
    center_coord + ref_point])

# pts_src = np.float32([
#     np.array([0., 0.]),
#     np.array([1., 1.]) * ref_point,
#     np.array([1., 0.]) * ref_point
# ])

print(f"pts_src: {pts_src}")
print(f"Shape of points in image space: {pts_src.shape}")
scatter_plot([pts_src], h, w)

## Convert points in image space into points in transformed image space

In [ ]:
# Random Affine
alpha = 50.
pts_dst1 = pts_src + np.array([alpha, alpha], dtype=pts_src.dtype)
scatter_plot([pts_src, pts_dst1,], h, w)

In [ ]:
def plot_images(imgs: List[Tensor]):
    """Generate PIL image for given list of tensors."""
    return ToPILImage()(
        make_grid([ToTensor()(img) for img in imgs], pad_value=1, padding=10))

In [ ]:
theta = np.radians(5)
c, s = np.cos(theta), np.sin(theta)
R = np.array(((c, -s), (s, c)))
print(R) 
pts_dst2 = (pts_src @ R.T).astype(pts_src.dtype)
scatter_plot([pts_src, pts_dst1, pts_dst2], h, w)

In [ ]:
# get transformation matrix
img = np.zeros((h, w, 3), dtype=np.uint8)
img = insert_grid(img, (25, 25), color=(255, 255, 255))

M1 = cv2.getAffineTransform(pts_src, pts_dst1)
trnsf_img1 = cv2.warpAffine(
    src=img,
    M=M1,
    dsize=(w, h), 
    borderMode=cv2.BORDER_REFLECT_101)

M2 = cv2.getAffineTransform(pts_src, pts_dst2)
trnsf_img2 = cv2.warpAffine(
    src=img,
    M=M2,
    dsize=(w, h), 
    borderMode=cv2.BORDER_REFLECT_101)

plot_images([img, trnsf_img1, trnsf_img2])

In [ ]:
M1.shape

In [ ]:
img = Image.open("/home/mittal.nit/projects/biomedical_image_segmentation/data/masks/train/0.tif")
img = np.expand_dims(np.array(img), axis=-1)
# trnsf_img1 = getAffineTransform(img, degree=0, alpha=50)
# plot_images([img, trnsf_img1])

In [ ]:
type(img)

In [ ]:
trnsf_img2 = getAffineTransform(img, degree=45, alpha=0)
plot_images([img, trnsf_img1, trnsf_img2])

In [ ]:
trnsf_img3 = getAffineTransform(img, degree=45, alpha=50)
plot_images([img, trnsf_img1, trnsf_img2, trnsf_img3])

# Elastic deformation

In [ ]:
from biomedical_image_segmentation.elastic_deform import (
    getElasticTransform,
    getAffineAndElasticDeform)

In [ ]:
img = np.zeros((h, w, 3), dtype=np.uint8)
img = insert_grid(img, (25, 25), color=(255, 255, 255))
plot_images([img])

In [ ]:
trnsf_img1 = getElasticTransform(img, sigma=12, alpha=100)
plot_images([img, trnsf_img1])

In [ ]:
trnsf_img2 = getAffineAndElasticDeform(
    img, 
    affine_transform=True, 
    affine_degree=45, 
    affine_alpha=10, 
    elastic_transform=True, 
    elastic_sigma=12, 
    elastic_alpha=120)
plot_images([img, trnsf_img1, trnsf_img2])

In [ ]:
img = Image.open("/home/mittal.nit/projects/biomedical_image_segmentation/data/masks/train/0.tif")
img = np.expand_dims(np.array(img), axis=-1)
seed = 40
trnsf_img1 = getAffineAndElasticDeform(
    img, 
    affine_transform=True, 
    affine_degree=10, 
    affine_alpha=0, 
    elastic_transform=True, 
    elastic_sigma=12, 
    elastic_alpha=200, 
    seed=seed)
plot_images([img, trnsf_img1,])

In [ ]:
trnsf_img2 = getAffineAndElasticDeform(
    img, 
    affine_transform=True, 
    affine_degree=10, 
    affine_alpha=0, 
    elastic_transform=True, 
    elastic_sigma=12, 
    elastic_alpha=200, 
    seed=seed)
plot_images([img, trnsf_img1, trnsf_img2])

In [ ]:
np.allclose(trnsf_img2, trnsf_img1)